In [16]:
!pip install requests-html
!pip install re
!pip install request
!pip install bs4

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request


In [17]:
!pip install lxml_html_clean

In [49]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from collections import deque
from urllib.parse import urlsplit
import pandas as pd
from requests_html import HTMLSession
import threading
import time
from tqdm import tqdm

In [54]:
#user_url = "https://www.thapar.edu/sitemap.xml"
#url = ["https://www.thapar.edu"]
# EMAIL_REGEX = r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"

EMAIL_REGEX = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

In [55]:
def get_urls_of_xml(xml_url):
    r = requests.get(xml_url)
    xml = r.text
    soup = BeautifulSoup(xml)

    links_arr = []
    for link in soup.find_all('loc'):
        linkstr = link.getText('', True)
        links_arr.append(linkstr)

    return links_arr

links_data_arr = get_urls_of_xml("https://www.ftb.ca.gov/sitemap.xml")
print(links_data_arr)

/tmp/ipython-input-439358939.py:4: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(xml)


['https://www.ftb.ca.gov/', 'https://www.ftb.ca.gov/index-es.html', 'https://www.ftb.ca.gov/file/index.html', 'https://www.ftb.ca.gov/file/after-you-file/amend-a-return/after-you-submit.html', 'https://www.ftb.ca.gov/file/after-you-file/amend-a-return/help-with-amended-returns.html', 'https://www.ftb.ca.gov/file/after-you-file/amend-a-return/index.html', 'https://www.ftb.ca.gov/file/after-you-file/amend-a-return/irs-changed-your-federal-tax-return.html', 'https://www.ftb.ca.gov/file/after-you-file/audit/disagree-with-an-npa.html', 'https://www.ftb.ca.gov/file/after-you-file/audit/index.html', 'https://www.ftb.ca.gov/file/after-you-file/audit/notice-of-proposed-assessment.html', 'https://www.ftb.ca.gov/file/after-you-file/audit/taxpayer-dispute-process-claim-for-refund.html', 'https://www.ftb.ca.gov/file/after-you-file/audit/taxpayer-dispute-process-npa.html', 'https://www.ftb.ca.gov/file/after-you-file/index.html', 'https://www.ftb.ca.gov/file/after-you-file/keeping-your-tax-records.ht

In [56]:
session = HTMLSession()

In [59]:
email=set()
for i in tqdm(links_data_arr):
        #requests.get(f'{i}', verify=False)
        r = session.get(i)
        for re_match in re.finditer(EMAIL_REGEX, r.html.raw_html.decode(errors="ignore")):
            email.add(((re_match.group())).replace("-",""))

100%|██████████| 2993/2993 [11:01<00:00,  4.53it/s]


In [60]:
df = pd.DataFrame(email)
df.to_csv('Emails.csv', index=False)